# Bayesian Database Search API Tutorial

## Preamble
Import necessary libraries.

In [ ]:
import requests
import math
import pandas

Set HTTP headers to be used across all requests.

In [ ]:
headers = {'content-type': 'application/json'}

Load visualization utility functions.

In [ ]:
%run visualize.py

## The data

In [ ]:
http_response = requests.get('http://bayesrest:5000/table-data', ())
assert http_response.status_code == 200

response_json = http_response.json()
data = response_json['data']

dataframe = pandas.DataFrame(
    data=data,
    columns=response_json['columns']
)
dataframe.index = dataframe['rowid']

def with_columns(rdf, columns=[]):
    return rdf.merge(
            dataframe.loc[:, ['rowid'] + columns], 
            on='rowid', 
            how='left'
    )

dataframe[:5]

## Columns most predictive of `Opioid_Deaths`

In [ ]:
column = 'Opioid_Deaths'

#### Fetch

In [ ]:
payload = dict(column=column)
response = requests.post(
    'http://bayesrest:5000/find-associated-columns',
    json=payload, 
    headers=headers
)
assert response.status_code == 200

#### Bar chart

In [ ]:
od_predictive_columns = response.json()
bar_chart = make_bar_chart(
    od_predictive_columns[:25],
    title='Columns most predictive of {}'.format(column), 
    x_axis='Relevance to {}'.format(column)
)
offline.iplot(bar_chart)

## Columns most predictive of `Total Property Crimes Rate (per 100000 Population)`

#### Fetch

In [ ]:
column = 'Total Property Crimes Rate (per 100000 Population)'
payload = dict(column=column)
response = requests.post(
    'http://bayesrest:5000/find-associated-columns',
    json=payload, 
    headers=headers
)
assert response.status_code == 200

#### Bar Chart

In [ ]:
columns = response.json()
bar_chart = make_bar_chart(
    columns[:25],
    title='Columns most predictive of {}'.format(column), 
    x_axis='Relevance to {}'.format(column)
)
offline.iplot(bar_chart)

## Counties with unlikely numbers of opioid deaths

### Unlikely counties without context

In [ ]:
target_column = 'Opioid_Deaths'

#### Fetch

In [ ]:
payload = {
    'target-column': target_column, 
    'context-columns': []
}
response = requests.post(
    'http://bayesrest:5000/find-anomalies', 
    json=payload, 
    headers={'content-type': 'application/json'}
)
assert response.status_code == 200

In [ ]:
df = pandas.DataFrame(response.json(), columns=['rowid', 'probability'])
df = df[df['probability'].notnull()]
df = with_columns(df, ['state_county_fips', 'Location', target_column])
df[:5]

#### Choropleth

In [ ]:
choropleth = make_choropleth(
    df['state_county_fips'], 
    df['probability'].transform(lambda p: math.log(p)), 
    title='Counties with unlikely values for {}'.format(target_column)
)
offline.iplot(choropleth)

#### Scatterplot

In [ ]:
scatter = make_scatterplot(
    xs=df[target_column],
    ys=df['probability'],
    text=df['Location'],
    x_axis=target_column, 
    y_axis='Probability Score'
)
offline.iplot(scatter)

### Unlikely counties in the context of predictively relevant columns

In [ ]:
target_column = 'Opioid_Deaths'

#### Fetch

In [ ]:
payload = {
    'target-column': target_column, 
    'context-columns': [
        '"Trump 2016"', 
        '"Total Population: Foreign Born: Not a Citizen"', 
        '"Families: Income in  below poverty level: Married Couple Family: with Related Child Living  Bellow Poverty Level"'
    ]
}
response = requests.post('http://bayesrest:5000/find-anomalies', json=payload, headers=headers)
assert response.status_code == 200

In [ ]:
df = pandas.DataFrame(response.json(), columns=['rowid', 'probability'])
df = df[df['probability'].notnull()]
df = with_columns(df, ['state_county_fips', 'Location', target_column])
df[:5]

#### Choropleth

In [ ]:
choropleth = make_choropleth(
    df['state_county_fips'], 
    df['probability'].transform(lambda p: math.log(p)), 
    title='''Anomalous counties in terms of {}, in the context of support for Trump, 
    <br>density of immigrants, and poverty'''.format(target_column)
)
offline.iplot(choropleth)

#### Scatterplot

In [ ]:
scatter = make_scatterplot(
    df[target_column],
    df['probability'],
    df['Location'],
    x_axis=target_column, 
    y_axis='Probability Score',
    title='''Anomalous counties in terms of {}, in the context of support for Trump, 
    <br> density of immigrants, and poverty'''.format(target_column)
)
offline.iplot(scatter)

### Unlikely counties in the context of the top 10 most relevant columns

In [ ]:
target_column = 'Opioid_Deaths'
context_columns = ['"{}"'.format(column['column']) for column in od_predictive_columns[1:11]]
context_columns

#### Fetch

In [ ]:
payload = {
    'target-column': target_column, 
    'context-columns': context_columns
}
response = requests.post('http://bayesrest:5000/find-anomalies', json=payload, headers=headers)
assert response.status_code == 200

In [ ]:
df = pandas.DataFrame(response.json(), columns=['rowid', 'probability'])
df = df[df['probability'].notnull()]
df = with_columns(df, ['state_county_fips', 'Location', target_column])
df[:5]

#### Choropleth

In [ ]:
choropleth = make_choropleth(
    df['state_county_fips'], 
    df['probability'].transform(lambda p: math.log(p)), 
    title='Anomalous counties in terms of {}, in the context of the top 10 most relevant variables'.format(target_column)
)
offline.iplot(choropleth)

#### Scatterplot

In [ ]:
scatter = make_scatterplot(
    df[target_column],
    df['probability'],
    df['Location'],
    
    x_axis=target_column, 
    y_axis='Probability Score',
    title='Anomalous counties in terms of {}, in the context of the top 10 most relevant columns'.format(target_column)
)
offline.iplot(scatter)

## Counties similar to a county with a low number of opioid deaths and low probability density of opioid deaths

In [ ]:
def location_to_rowid(location):
    rowid = df.loc[df['Location'] == location]['rowid'].iloc[0]
    return numpy.asscalar(rowid)

In [ ]:
context_column = 'Opioid_Deaths'
location = 'Presidio County, Texas'
rowid = location_to_rowid(location)
assert rowid is not None

#### Fetch

In [ ]:
payload = {
    'target-row': rowid, 
    'context-column': context_column
}

response = requests.post(
    'http://bayesrest:5000/find-peers', 
    json=payload, 
    headers=headers)
assert response.status_code == 200

In [ ]:
df = pandas.DataFrame(response.json(), columns=['rowid', 'similarity'])
df = df[df['similarity'].notnull()]
df = with_columns(df, ['state_county_fips', 'Location', context_column])
df[:5]

#### Choropleth

In [ ]:
choropleth = make_choropleth(
    fips=df['state_county_fips'], 
    values=df['similarity'], 
    legend_title='Similarity Score',
    title='Counties similar to {} with respect to {}'.format(location, context_column),
    color_scale=["#f7fbff","#ebf3fb","#deebf7","#d2e3f3","#c6dbef","#b3d2e9","#9ecae1", "#85bcdb","#6baed6","#57a0ce","#4292c6"]
)
offline.iplot(choropleth)

#### Scatterplot

In [ ]:
scatter = make_scatterplot(
    xs=df[context_column],
    x_axis=context_column, 
    ys=df['similarity'],
    y_axis='Similarity Score',
    text=df['Location'],
    title='Counties similar to {} with respect to {}'.format(location, context_column)
)
offline.iplot(scatter)

## Counties similar to a county with a low number of opioid deaths and high probability density of opioid deaths

#### Fetch

In [ ]:
context_column = 'Opioid_Deaths'
location = 'Starr County, Texas'
rowid = location_to_rowid(location)
assert rowid is not None

In [ ]:
payload = {
    'target-row': rowid, 
    'context-column': context_column
}

response = requests.post(
    'http://bayesrest:5000/find-peers', 
    json=payload, 
    headers=headers)
assert response.status_code == 200

In [ ]:
df = pandas.DataFrame(response.json(), columns=['rowid', 'similarity'])
df = df[df['similarity'].notnull()]
df = with_columns(df, ['state_county_fips', 'Location', context_column])
df[:5]

#### Choropleth

In [ ]:
choropleth = make_choropleth(
    fips=df['state_county_fips'], 
    values=df['similarity'], 
    legend_title='Similarity Score',
    title='Counties Similar to {} with respect to {}'.format(location, context_column),
    color_scale=["#f7fbff","#ebf3fb","#deebf7","#d2e3f3","#c6dbef","#b3d2e9","#9ecae1", "#85bcdb","#6baed6","#57a0ce","#4292c6"]
)
offline.iplot(choropleth)

#### Scatterplot

In [ ]:
scatter = make_scatterplot(
    xs=df[context_column],
    x_axis=context_column, 
    ys=df['similarity'],
    y_axis='Similarity Score',
    text=df['Location'],
    title='Counties Similar to {} with respect to {}'.format(location, context_column)
)
offline.iplot(scatter)

## Counties similar to a county with a high number of opioid deaths

Here we'll focus on counties similar to a county with a high number of opioid deaths: <b>Los Angeles County, California.</b>

#### Fetch

In [ ]:
context_column = 'Opioid_Deaths'
location = 'Los Angeles County, California'
rowid = location_to_rowid(location)
assert rowid is not None

In [ ]:
payload = {
    'target-row': rowid,
    'context-column': context_column
}

response = requests.post(
    'http://bayesrest:5000/find-peers', 
    json=payload, 
    headers=headers)
assert response.status_code == 200

In [ ]:
df = pandas.DataFrame(response.json(), columns=['rowid', 'similarity'])
df = df[df['similarity'].notnull()]
df = with_columns(df, ['state_county_fips', 'Location', context_column])
df[:5]

#### Choropleth

In [ ]:
choropleth = make_choropleth(
    fips=df['state_county_fips'], 
    values=df['similarity'], 
    legend_title='Similarity Score',
    title='Counties similar to {} with respect to {}'.format(location, context_column),
    color_scale=["#f7fbff","#ebf3fb","#deebf7","#d2e3f3","#c6dbef","#b3d2e9","#9ecae1", "#85bcdb","#6baed6","#57a0ce","#4292c6"]
)
offline.iplot(choropleth)

#### Scatterplot

In [ ]:
scatter = make_scatterplot(
    xs=df[context_column],
    x_axis=context_column, 
    ys=df['similarity'],
    y_axis='Similarity Score',
    text=df['Location'],
    title='Counties Similar to {} with respect to {}'.format(location, context_column)
)
offline.iplot(scatter)